In [2]:
import statsmodels.formula.api as smf

from utils import data_utils as du
from utils import stats_utils as su

In [3]:
POLICY_YEAR: int = 2019
START_YEAR: int = 2016
END_YEAR: int = 2023

energy_data = du.load_data()
concurrent_df = du.concurrent_buildings(
    input_df=energy_data, start_year=2016, end_year=2023
)
did_df = su.prepare_did_data(concurrent_df)

did_df["Post"] = (did_df["Data Year"] >= POLICY_YEAR).astype(int)
median_rating = did_df["Chicago Energy Rating"].median(skipna=True)
did_df["LowRating"] = (did_df["Chicago Energy Rating"] <= median_rating).astype(int)
median_score = did_df["ENERGY STAR Score"].median(skipna=True)
did_df["LowScore"] = (did_df["ENERGY STAR Score"] <= median_score).astype(int)

# DiD Model Revisit

### Water Usage as Outcome

$$
{Water Usage}_{it} =
\alpha
+ \beta_1 \text{Post}_{t}
+ \beta_2 \text{LowRating}_{i}
+ \beta_3 (\text{Post}_{t} \times \text{LowRating}_{i})
+ \gamma_1 \ln(\text{FloorArea}_{i})
+ \gamma_2 \text{PropertyType}_{i}
+ \gamma_3 \text{Year}_{t}
+ \gamma_4 \text{YearBuilt}_{t}
+ \varepsilon_{it}
$$

where  

- $ \text{WaterUsage}_{it} $ : total water use (kGal) for building i in year t

- $ \text{Post}_{t}=1 $ : if $t \ge 2019$ (post-placard period), 0 otherwise

- $ \text{LowRating}_{i}=1 $ : if the building’s initial Chicago Energy Rating ≤ 2 (treated group), 0 otherwise

- $ \text{Post}{t}\times\text{LowRating}{i} $ : interaction capturing the differential change for treated buildings after the policy

- $ \ln(\text{FloorArea}{i}) $, $ \text{PropertyType}{i} $, $ \text{Year}{t} $, $ \text{YearBuilt}{i} $ : control variables for building size, type, time, and age effects

- $ \varepsilon_{it} $ : error term


In [4]:
model_water = su.run_did_regression(did_df, "Water Use (kGal)", include_data_year=True)
su.summarize_did_results(model_water)

/project/.venv/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 56, but rank is 45
  warnings.warn('covariance of constraints does not have full '


,coef,std_err,p_value,Significance
C(Q('Primary Property Type'))[T.fitness center/health club/gym],9049.2184,24028.3568,0.7065,
C(Q('Primary Property Type'))[T.hospital (general medical & surgical)],-32584.7216,41299.1776,0.4301,
C(Q('Primary Property Type'))[T.laboratory],-3755.3544,26606.5757,0.8878,
C(Q('Primary Property Type'))[T.other - specialty hospital],-43189.6002,43354.0407,0.3191,
Post,42256.6176,26563.6682,0.1117,
LowRating,-10781.0655,11961.5180,0.3674,
Interaction,136636.1308,176862.0726,0.4398,
ln_FloorArea,28836.5022,15288.1377,0.0593,*


### Pre-trend

In [5]:
did_df["PrePeriod"] = did_df["Data Year"] < POLICY_YEAR

formula_pretrend = """
Q('Site EUI (kBtu/sq ft)') ~ LowRating
+ C(Q('Data Year')):LowRating
+ C(Q('Data Year'))
+ ln_FloorArea
+ C(Q('Primary Property Type'))
"""

model_pretrend = smf.ols(
    formula=formula_pretrend, data=did_df[did_df["Data Year"] < POLICY_YEAR]
).fit(cov_type="HC1")

su.summarize_did_results(model_pretrend)

/project/.venv/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 46, but rank is 38
  warnings.warn('covariance of constraints does not have full '


,coef,std_err,p_value,Significance
C(Q('Primary Property Type'))[T.fitness center/health club/gym],69.7390,36.7352,0.0576,*
C(Q('Primary Property Type'))[T.hospital (general medical & surgical)],203.7522,61.3102,0.0009,***
C(Q('Primary Property Type'))[T.laboratory],277.3577,54.3432,0.0000,***
C(Q('Primary Property Type'))[T.other - specialty hospital],90.3975,34.8554,0.0095,***
LowRating,40.4474,2.7915,0.0000,***
ln_FloorArea,-2.4179,1.0192,0.0177,**


Baseline Heterogeneity
- Pre-policy, low-rated buildings already used about 40 kBtu/sq ft more energy than higher-rated buildings, indicating a existed energy gap
- Hospitals, Labs, and fitness centers consume more energy than others

In [6]:
formula_posttrend = """
Q('Site EUI (kBtu/sq ft)') ~ YearsAfter * LowRating
+ ln_FloorArea
+ C(Q('Primary Property Type'))
"""

df_post = did_df[did_df["Data Year"] >= POLICY_YEAR].copy()
df_post["YearsAfter"] = df_post["Data Year"] - POLICY_YEAR

model_posttrend = smf.ols(formula=formula_posttrend, data=df_post).fit(cov_type="HC1")

su.summarize_did_results(model_posttrend)

/project/.venv/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 51, but rank is 47
  warnings.warn('covariance of constraints does not have full '


,coef,std_err,p_value,Significance
C(Q('Primary Property Type'))[T.fitness center/health club/gym],23.4406,14.9222,0.1162,
C(Q('Primary Property Type'))[T.hospital (general medical & surgical)],115.0371,12.2170,0.0000,***
C(Q('Primary Property Type'))[T.laboratory],212.0463,24.1370,0.0000,***
C(Q('Primary Property Type'))[T.other - specialty hospital],79.5473,12.5818,0.0000,***
LowRating,38.6357,1.9092,0.0000,***
YearsAfter:LowRating,-2.7691,0.5929,0.0000,***
ln_FloorArea,-3.8931,0.3892,0.0000,***


### DDD

$$
Y_{itc} =
\alpha
+ \beta_1 \text{Post}_t
+ \beta_2 \text{LowRating}_i
+ \beta_3 \text{COVIDGroup}_c
+ \beta_4 (\text{Post}_t \times \text{LowRating}_i)
+ \beta_5 (\text{Post}_t \times \text{COVIDGroup}_c)
+ \beta_6 (\text{LowRating}_i \times \text{COVIDGroup}_c)
+ \beta_7 (\text{Post}_t \times \text{LowRating}_i \times \text{COVIDGroup}_c)
+ \gamma_1 \ln(\text{FloorArea}_i)
+ \gamma_2 \text{PropertyType}_i
+ \gamma_3 \text{Year}_t
+ \varepsilon_{itc}
$$

In [7]:
did_df["COVIDGroup"] = (
    did_df["Primary Property Type"]
    .isin(["Office", "Retail Store", "College/University"])
    .astype(int)
)

formula_ddd = """
Q('Site EUI (kBtu/sq ft)') ~ Post + LowRating + COVIDGroup
+ Post:LowRating + Post:COVIDGroup + LowRating:COVIDGroup
+ Post:LowRating:COVIDGroup
+ ln_FloorArea + C(Q('Data Year')) + C(Q('Primary Property Type'))
"""

model_ddd = smf.ols(formula=formula_ddd, data=did_df).fit(cov_type="HC1")
su.summarize_did_results(model_ddd)

/project/.venv/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 60, but rank is 1
  warnings.warn('covariance of constraints does not have full '


,coef,std_err,p_value,Significance
C(Q('Primary Property Type'))[T.fitness center/health club/gym],3.311220e+01,1.454090e+01,0.0228,**
C(Q('Primary Property Type'))[T.hospital (general medical & surgical)],1.343444e+02,1.682150e+01,0.0000,***
C(Q('Primary Property Type'))[T.laboratory],2.241094e+02,2.256860e+01,0.0000,***
C(Q('Primary Property Type'))[T.other - specialty hospital],8.345040e+01,1.291060e+01,0.0000,***
Post,-5.511371e+11,5.536246e+11,0.3195,
LowRating,3.940620e+01,2.405000e+00,0.0000,***
Post:LowRating,-6.803700e+00,2.490900e+00,0.0063,***
Post:COVIDGroup,0.000000e+00,0.000000e+00,NaN,
LowRating:COVIDGroup,0.000000e+00,0.000000e+00,NaN,
Post:LowRating:COVIDGroup,0.000000e+00,0.000000e+00,NaN,


Hardcoded building types to isolate COVID effects; covid terms not significant, meaning less impacts from COVID.

## Predictive Modeling - prelimary exploration

Prediction Goal:
- Binary Outcome:
        If the building's energy usage decrease by cetain amount post policy
- Predictors: Building Charactistics (floor areas, years of built, etc.), Key Metrics, Geographical Locations, and Baseline metrics (rating/score)

In [8]:
did_df["Improved_EUI"] = (
    did_df.groupby("ID")["Site EUI (kBtu/sq ft)"].diff() < 0
).astype(int)

did_df = did_df.dropna(subset=["Improved_EUI"])

features = [
    "Gross Floor Area - Buildings (sq ft)",
    "Year Built",
    "Chicago Energy Rating",
    "ENERGY STAR Score",
    "Water Use (kGal)",
    "GHG Intensity (kg CO2e/sq ft)",
]

X = did_df[features].fillna(0)
y = did_df["Improved_EUI"]

Next Stepts:
After resolving the concurrent shock issue, may run Logistic, Random Forest, and CV.